In [3]:
%matplotlib notebook

# Import external libraries to present an manipulate the data
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Import docplex model to generate the problem to optimize
from docplex.mp.model import Model

# Import the libraries needed to employ the QAOA quantum algorithm using OpenQAOA
from openqaoa import QAOA

# method to covnert a docplex model to a qubo problem
from openqaoa.problems.converters import FromDocplex2IsingModel #check this method and properties
from openqaoa.backends import create_device

# method to find the correct states for the QAOA object 
from openqaoa.utilities import ground_state_hamiltonian

ImportError: Missing optional 'qiskit' dependencies. To install run: pip install azure-quantum[qiskit]

In [ ]:
# Datos del problema de Prueba



In [ ]:
def Problem(values, weights, max_weight):
    
    # initialize a model
    mdl = Model(name="Knaspack")
    # indicate the binary variables 
    x = {i: mdl.binary_var(name=f"x_{i}") for i in range(len(values))}  
    # define the objective function
    mdl.minimize(mdl.sum(values[i] * x[i] for i in x))    
    # add the constraints
    mdl.add_constraint(mdl.sum(weights[i] * x[i] for i in x) <= max_weight)
    return  FromDocplex2IsingModel(mdl)

In [ ]:
problem =  Problem(values, weights, max_weight)

# Ising encoding of the QUBO problem for binpacking problem
qubo = problem.ising_model

# Docplex encoding of the QUBO problem for binpacking problem
mdl_qubo_docplex = problem.qubo_docplex

mdl_qubo_docplex.prettyprint()

In [ ]:
def funcionesQAOA(device,tipo,ham):
    qaoa = QAOA()
    qaoa.set_device(device)

    # Set the parameters to use the QAOA algorithm
    qaoa.set_backend_properties(n_shots=1024, seed_simulator=1)
    qaoa.set_circuit_properties(p=1, param_type='standard', init_type=tipo, mixer_hamiltonian=ham)

    qaoa.compile(qubo)

    # Run the QAOA algorithm
    qaoa.optimize()
    return qaoa.result

In [ ]:
# Initialize the QAOA object and use a device
tipos = ['rand','ramp']
ham = ['xy','x']
dispositivos = ['qiskit.statevector_simulator','qiskit.qasm_simulator']
for k in dispositivos:
    for i in tipos:
        for j in ham:
            device = create_device("local", k)
            results_sv = funcionesQAOA(device,i,j)
            print("Dispositivo: ",k, " Tipo: ",i, " Mixer: ",j)
            print(pd.DataFrame(results_sv.lowest_cost_bitstrings(5)))

In [ ]:
correct_solution = ground_state_hamiltonian(qaoa.cost_hamil)
print(correct_solution)